In [1]:
from requests import Timeout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import spotipy as spy
import pickle
import csv


In [2]:
with open('secret.txt') as s:
    client_secret = next(s)
s.close()
client_id = '339e9f02e5254f07a945aef3f9471a31'

In [3]:
credentials_manager = spy.oauth2.SpotifyClientCredentials(client_id, client_secret)
sp = spy.Spotify(client_credentials_manager=credentials_manager)

In [4]:
url = input("Enter URL for the PUBLIC playlist you want to inspect: ")
user = input("Enter the username of the playlist's owner: ")
splitted = url.split("/")
URI = splitted[len(splitted)-1]
print(URI)

4b9mSnY53QnAf6KxFffw2h


In [5]:
pl_t = sp.user_playlist_tracks(user, URI)
tmp = pl_t['items']
ids = []
while pl_t['next']:
    pl_t = sp.next(pl_t)
    tmp.extend(pl_t['items'])
    for s in tmp:
        ids.append(s["track"]["id"])

In [6]:
i = len(ids)
print(i)
if len(ids) != len(set(ids)):
    ids = list(set(ids))
    print("Duplicates found, {} rows cleaned.".format(i-len(ids)))
    print(len(ids))
else:
    print("No duplicates")

154
Duplicates found, 2 rows cleaned.
152


In [7]:
di = {"name": [], "lenght": [], "popularity": [], "acousticness": [], "danceability": [], 
"energy": [], "instrumentalness": [], "liveness": [], "loudness": [], "speechiness": [],
"tempo": [], "time_signiature": []}

for i in range(len(ids)):
    try:
        metadata = sp.track(ids[i])
        features = sp.audio_features(ids[i])
        di["name"].append(metadata['name'])
        di['lenght'].append(metadata['duration_ms'])
        di['popularity'].append(metadata['popularity'])
            
        di['acousticness'].append(features[0]['acousticness'])
        di['danceability'].append(features[0]['danceability'])
        di['energy'].append(features[0]['energy'])
        di['instrumentalness'].append(features[0]['instrumentalness'])
        di['liveness'].append(features[0]['liveness'])
        di['loudness'].append(features[0]['loudness'])
        di['speechiness'].append(features[0]['speechiness'])
        di['tempo'].append(features[0]['tempo'])
        di['time_signiature'].append(features[0]['time_signature'])
    except spy.SpotifyException:
        print("Spotify error at id", ids[i])
    except Timeout:
        print("API not responding, trying again.")
        if (i >= 1):
            i = i - 1
        else:
            i = 0
        continue
ddf = pd.DataFrame(di)
ddf.head()
     

HTTP Error for GET to https://api.spotify.com/v1/tracks/1r8mABSC4rx9CMNIC8rCFi with Params: {'market': None} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/tracks/3Zq9BbjALjRAwoO3PvzUwW with Params: {'market': None} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/tracks/7BGprWocWmRdWww8709Rd2 with Params: {'market': None} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/tracks/0JKSDo09ciAiKKQl35FevJ with Params: {'market': None} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/tracks/2Ot6UNQ2H25L18Ll0b7yCV with Params: {'market': None} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/tracks/0aD5NqlDu8eY7TT4F8wBj2 with Params: {'market': None} returned 404 due to None


,name,lenght,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signiature
0,Tahroja paperilla,286720,47,0.00799,0.763,0.798,0.000000,0.0449,-6.436,0.0323,120.274,4
1,Levottomat Jalat,190013,42,0.09030,0.543,0.890,0.000004,0.0951,-5.398,0.1070,93.962,4
2,Vie Mut Minne Vaan,228160,49,0.06960,0.422,0.747,0.000000,0.1020,-8.550,0.0290,163.213,4
3,Haltin häät,174826,35,0.64400,0.540,0.612,0.000000,0.1080,-6.929,0.0345,96.139,4
4,Totuutta ja tehtävää,242279,41,0.00670,0.541,0.700,0.011900,0.5680,-7.093,0.0265,94.981,4


In [8]:
pop_pred = pickle.load(open("PopularityPred.sav", "rb"))
class_pred = pickle.load(open("LabelPred.sav", "rb"))

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
x = ddf.copy().drop(columns=['popularity', 'name'])
pop = pop_pred.predict(x)
print(pop)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [10]:
x = ddf.copy().drop(columns=["name"])
s = MinMaxScaler().fit_transform(x)
n_df = pd.DataFrame(s)
cl = pd.Series(class_pred.predict(n_df))
y = ddf.copy()
y["labels"] = cl
print(y[["name", "labels"]])

                                              name  labels
0                                Tahroja paperilla       0
1                                 Levottomat Jalat       0
2                               Vie Mut Minne Vaan       0
3                                      Haltin häät       0
4                             Totuutta ja tehtävää       0
..                                             ...     ...
141                               Elän Vain Kerran       0
142                                          Ikävä       0
143                         En oo käyny Irlannissa       0
144                      Unelmia ja toimistohommia       0
145  Mä Elän Vieläkin - Highwayman / 2006 Remaster       0

[146 rows x 2 columns]


In [15]:
new_df = pd.read_csv("Labelresults.csv", index_col=0)
new_df.head()

,name,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,labels
0.0,RAP DEVIL,286117,68,0.217,0.895,0.8390000000000001,3e-06,0.239,-4.512,0.22699999999999998,149.951,4,-1
1.0,Another Day in Paradise - 2016 Remaster,322133,78,0.736,0.78,0.565,0.0026899999999999997,0.063,-7.32,0.0306,101.96700000000001,4,-1
2.0,Jamming,211000,77,0.102,0.934,0.467,3.98e-05,0.109,-10.99,0.307,123.583,4,-1
3.0,Mo Money Mo Problems (feat. Puff Daddy & Mase)...,257399,79,0.0128,0.845,0.884,2.1000000000000002e-06,0.203,-4.506,0.0759,104.536,4,0
4.0,Drive - 2017 Remaster,234332,72,0.115,0.261,0.703,0.00078,0.102,-7.871,0.0414,83.495,4,-1


In [27]:
song = input("Give a song you want to compare:" )
label = y.loc[y["name"] == song]["labels"]
label = label.values[0]
print("Lets check out some song with similar characteristics for the label ", label, "!")
print(new_df[new_df["labels"] == 0])

Lets check out some song with similar characteristics for the label  0 !
Empty DataFrame
Columns: [name, length, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, labels]
Index: []
